[View in Colaboratory](https://colab.research.google.com/github/ChristofHenkel/household_image_classification/blob/master/Xception_LSTM.ipynb)

## Installing requirements

In [11]:
VALID_ID = '1P45QJN_Yd2PKFCE_9922jdvVVDd-XIyQ'
CHUNK_IDS = ['1P6YTpC6ioXze2cAB0Rz2umxGd27DxtO4',
             '1DXVqFuVn51Qa64qNfHfCSruRRfrl8MN6',
             '1EjC0thAMzsP815v5647tBMVMyVlgerwr',
             '1A_kHfHOn8EhEBNsGUniUe0IkNYKXImA7',
             '1zRVyfi7l5nGNCLimjorW03gk2VhB_LAo',
             '1H1sQPWUUTjPWGtcN5EJYKO_rmYVYH1KL'
            ]
CLIENT_SECRETS_ID = '1sShAACG19QKYvFk5fuD5DzmWzrKtF11-'
CALLBACKS_ID = '19GCUwlQU9ofdTqsgPrT0oC3djOtn-nGr'
YAML_FILE_ID = '1wQshZQ-tywmBWFlhAbc5WFPn9HrCIgOF'
BEST_MODEL_ID = '11SqL318CF7NV5J-YqRJsueIVW8OQltu7'
!pip install -U -q PyDrive
#!pip install keras
!pip install tqdm
#!mkdir train
#!mkdir valid

import os
import gzip
import pickle
import random
import shutil
import zipfile
import time
from tqdm import tqdm
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [3]:
!ls

datalab  valid


In [0]:
#file = drive.CreateFile({'id': CLIENT_SECRETS_ID})
#file.GetContentFile('client_secrets.json')
settings = drive.CreateFile({'id': YAML_FILE_ID})
settings.GetContentFile('settings.yaml')

In [0]:
valid_file = drive.CreateFile({'id': VALID_ID})
valid_file.GetContentFile('valid.zip')
!unzip valid.zip
!rm valid.zip

In [0]:
def exchange_train_chunk(chunk_ind):
  print('exchange train chunk with chunk %s'%chunk_ind)
  if os.path.exists("train"):
    shutil.rmtree('train')
  train_file = drive.CreateFile({'id': CHUNK_IDS[chunk_ind]})
  train_file.GetContentFile('train.zip')
  with zipfile.ZipFile("train.zip","r") as zip_ref:
    zip_ref.extractall('train')
  os.remove('train.zip')

In [0]:
tic = time.time()
exchange_train_chunk(0)
toc = time.time()
print(toc-tic)

exchange train chunk with chunk 0
73.4421055316925


In [0]:
from keras.applications.xception import Xception
from keras.layers import Dropout, Dense, TimeDistributed, SpatialDropout1D, Bidirectional, CuDNNLSTM
from keras.metrics import top_k_categorical_accuracy
from keras.callbacks import EarlyStopping,ModelCheckpoint, Callback,ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

def top1_loss(y_true,y_pred):
    return 1- top_k_categorical_accuracy(y_true,y_pred,k=1)


In [6]:
BATCH_SIZE = 16

train_data_gen = ImageDataGenerator(rescale=1./255,
                                    vertical_flip=True,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.2,
                                    shear_range=0.2
                                    )

valid_data_gen = ImageDataGenerator(rescale=1./255,
                                    
                                    )
valid_generator = train_data_gen.flow_from_directory(directory='valid',
                             target_size=(224,224),
                            batch_size=BATCH_SIZE,
                             class_mode='categorical')

Found 6309 images belonging to 128 classes.


In [0]:

base_model = Xception(weights='imagenet', include_top=False,input_shape=(224,224,3))
inp = base_model.output
main = TimeDistributed(Bidirectional(CuDNNLSTM(256)))(inp)
main = SpatialDropout1D(0.4)(main)
main = Bidirectional(CuDNNLSTM(256))(main)
main = Dropout(0.4)(main)
predictions = Dense(128,activation='softmax',kernel_regularizer=regularizers.l2(0.0001))(main)


model = Model(inputs=base_model.input, outputs=predictions)


#model.summary()
model.compile(optimizer=Adam(lr = 0.00003), loss='categorical_crossentropy',metrics=[top1_loss])


In [0]:
check_point = ModelCheckpoint('best_model.hdf5', monitor="val_loss", mode="min", save_best_only=True, verbose=1)
early_stop = EarlyStopping(patience=4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=2, min_lr=0)

In [0]:
init = model.fit_generator(valid_generator,
                        #steps_per_epoch=train_generator.classes.size//BATCH_SIZE,
                        steps_per_epoch=1,
                        epochs=1,
                        validation_data=valid_generator,
                        validation_steps=1,
                        verbose=1,
                        callbacks = [check_point,early_stop,reduce_lr]
                       )

Epoch 1/1
1/1 [==============================] - 11s 11s/step - loss: 5.4070 - top1_loss: 1.0000 - val_loss: 5.0684 - val_top1_loss: 1.0000

Epoch 00001: val_loss improved from inf to 5.06837, saving model to best_model.hdf5


In [0]:
best_model = drive.CreateFile({'id': BEST_MODEL_ID})
best_model.GetContentFile('best_model.hdf5')
callbacks = drive.CreateFile({'id': CALLBACKS_ID})
callbacks.GetContentFile('callbacks.p')

In [0]:
model.load_weights('best_model.hdf5')

In [0]:
def save_callbacks():
  cb_dict = {'check_point':{'epochs_since_last_save':check_point.epochs_since_last_save,
                            'best':check_point.best},
            'early_stop':{'best':early_stop.best,'wait':early_stop.wait},
            'reduce_lr':{'lr':float(K.get_value(model.optimizer.lr))}}
  with open('callbacks.p','wb') as f:
    pickle.dump(cb_dict,f)
    
def load_callbacks():
  with open('callbacks.p','rb') as f:
    cb_dict = pickle.load(f)
  check_point.best = cb_dict['check_point']['best']
  check_point.epochs_since_last_save = cb_dict['check_point']['epochs_since_last_save']
  early_stop.best = cb_dict['early_stop']['best']
  early_stop.wait = cb_dict['early_stop']['wait']
  K.set_value(model.optimizer.lr,cb_dict['reduce_lr']['lr'])

In [0]:
load_callbacks()
  

In [0]:
!ls

best_model.hdf5  callbacks.p  datalab  settings.yaml  train  valid


In [0]:
histories = []
for epoch in range(30):
  for c,chunk_id in enumerate(CHUNK_IDS):
    if gauth.access_token_expired:
    # Refresh them if expired
      gauth.Refresh()
    exchange_train_chunk(c)
    train_generator = train_data_gen.flow_from_directory(directory='train/chunk%s'%c +'/',
                             target_size=(224,224),
                            batch_size=BATCH_SIZE,
                             class_mode='categorical')
    model.fit_generator(train_generator,
                        steps_per_epoch=train_generator.classes.size//BATCH_SIZE,
                        #steps_per_epoch=100,
                        epochs=1,
                        validation_data=valid_generator,
                        validation_steps=valid_generator.classes.size//BATCH_SIZE,
                        #validation_steps=10,
                        verbose=1,
                        callbacks = [check_point,early_stop,reduce_lr]
                       )
    save_callbacks()
    if check_point.epochs_since_last_save == 0:
      if gauth.access_token_expired:
        # Refresh them if expired
        gauth.Refresh()
      uploaded = drive.CreateFile({'id': BEST_MODEL_ID})
      uploaded.SetContentFile('best_model.hdf5')
      print('uploading model')
      uploaded.Upload()
      uploaded = drive.CreateFile({'id': CALLBACKS_ID})
      uploaded.SetContentFile('callbacks.p')
      print('uploading callback data')
      uploaded.Upload()

exchange train chunk with chunk 0
Found 28000 images belonging to 128 classes.
Epoch 1/1
 557/1750 [========>.....................] - ETA: 14:00 - loss: 4.7031 - top1_loss: 0.9440

1749/1750 [============================>.] - ETA: 0s - loss: 3.8466 - top1_loss: 0.8119

1750/1750 [==============================] - 1383s 791ms/step - loss: 3.8458 - top1_loss: 0.8118 - val_loss: 2.6719 - val_top1_loss: 0.6401

Epoch 00001: val_loss improved from inf to 2.67188, saving model to best_model.hdf5
uploading model
uploading callback data
exchange train chunk with chunk 1
Found 28000 images belonging to 128 classes.
Epoch 1/1
 183/1750 [==>...........................] - ETA: 18:02 - loss: 2.6440 - top1_loss: 0.6226

1749/1750 [============================>.] - ETA: 0s - loss: 2.3134 - top1_loss: 0.5626

1750/1750 [==============================] - 1373s 785ms/step - loss: 2.3132 - top1_loss: 0.5626 - val_loss: 1.9251 - val_top1_loss: 0.5010

Epoch 00001: val_loss improved from 2.67188 to 1.92507, saving model to best_model.hdf5
uploading model
uploading callback data
exchange train chunk with chunk 2
Found 28000 images belonging to 128 classes.
Epoch 1/1
 181/1750 [==>...........................] - ETA: 17:58 - loss: 2.0270 - top1_loss: 0.5148

1749/1750 [============================>.] - ETA: 0s - loss: 1.8477 - top1_loss: 0.4691

1750/1750 [==============================] - 1374s 785ms/step - loss: 1.8477 - top1_loss: 0.4691 - val_loss: 1.6166 - val_top1_loss: 0.4326

Epoch 00001: val_loss improved from 1.92507 to 1.61662, saving model to best_model.hdf5
uploading model
uploading callback data
exchange train chunk with chunk 3
Found 28000 images belonging to 128 classes.
Epoch 1/1
 181/1750 [==>...........................] - ETA: 18:05 - loss: 1.6974 - top1_loss: 0.4454

1520/1750 [=========================>....] - ETA: 2:38 - loss: 1.6331 - top1_loss: 0.4229

TODOS:
Continue training:
  model file -> easy

*   save callbacks
*   save history

